In [33]:
import os
import pickle
import numpy as np

import tensorflow as tf
import tensorflow.keras.layers as layers
from tensorflow.keras.utils import Sequence

import Levenshtein as Lev
import sys

In [3]:
print(tf.__version__)

2.0.0


In [4]:
def wer(s1, s2):

    s1 =s1.lower()
    s2 =s2.lower()
    b = set(s1.lower().split() + s2.lower().split())
    
    word2char = dict(zip(b, range(len(b))))


    w1 = [chr(word2char[w]) for w in s1.split()]
    w2 = [chr(word2char[w]) for w in s2.split()]
    return Lev.distance(''.join(w1), ''.join(w2))/float(len(s2.split()))

To Decide
1. Batch Norm vs Layer Norm or No Norm for attention model
2. Autoencoder (CNN vs Vanilla vs None)
3. Shuffle or No

In [5]:
def autoencoder():
    model = tf.keras.Sequential()
    model.add(layers.Dense(350))
    model.add(layers.Dropout(rate = 0.2))
    model.add(layers.Dense(200))
    model.add(layers.Dropout(rate = 0.2))
    model.add(layers.Dense(100))
    model.add(layers.Dropout(rate = 0.2))
    model.add(layers.Dense(200))
    model.add(layers.Dropout(rate = 0.2))
    model.add(layers.Dense(350))
    model.add(layers.Dropout(rate = 0.2))
    model.add(layers.Dense(513))    
    return model

In [6]:
model = autoencoder()

In [13]:
train_path = "./LibriSpeech100/train/train_all/"
dev_path = "./LibriSpeech100/dev/dev_all/"
test_path = "./LibriSpeech100/test/test_all/"

In [21]:
class DataGenerator(Sequence):
    def __init__(self, path, to_fit = True):
        self.path = path
        self.list_X, self.list_Y = self.getLists()
        self.to_fit = to_fit
    
    def __len__(self):
        return len(self.list_X)
    
    def __getitem__(self, index):      
        dict_X = self.get_dict_X(index)   
        
        if self.to_fit:
            dict_Y = self.get_dict_Y(index)
            X, Y = self.generate_XY(dict_X, dict_Y)
            return X, Y
        
        X = self.generate_X(dict_X)
        return X
    
    def getLists(self):
        list_X = []
        list_Y = []
        for item in sorted(os.listdir(self.path)):
            ext = item.split(".")[-1]
            if ext == 'pkl':
                list_X.append(item)
            elif ext == 'txt':
                list_Y.append(item)
        return list_X, list_Y
    
    def get_dict_X(self, index):
        file_name = self.path + self.list_X[index]
        with open(file_name, 'rb') as pickle_file:
            dict_X = pickle.load(pickle_file)
        return dict_X
    
    def get_dict_Y(self, index):
        file = open(train_path+filename)
        dict_Y = {}
        for line in file:
            data = line.split()
            key = data[0]
            value = ' '.join(data[1:])
            dict_Y[key] = value
        return dict_Y
    
    def generate_XY(dict_X, dict_Y):
        X = []
        Y = []
        for key in dict_X:
            X.append(dict_X[key])
            Y.append(dict_Y[key])
        return X, Y
    
    def generate_X(dict_X):
        X = [value for key, value in dict_X.items()]
        return X

In [34]:
list_X = []
list_Y = []
for item in sorted(os.listdir(train_path)):
    ext = item.split(".")[-1]
    if ext == 'pkl':
        list_X.append(item)
    elif ext == 'txt':
        list_Y.append(item)

In [35]:
for filename in list_Y:
    file = open(train_path+filename)
    dict_Y = {}
    for line in file:
        data = line.split()
        key = data[0]
        value = ' '.join(data[1:])
        dict_Y[key] = value

In [39]:
Y = [value for key, value in dict_Y.items()]

In [40]:
Y


['AND FOR THIS THE VILLAGE BOYS WITH WHOM HE PLAYED USED TO QUARREL WITH HIM SAYING FATHERLESS ORPHAN WHY DO YOU CHEAT SO ONE DAY HE ASKED HIS MOTHER WHY THEY CALLED HIM THAT NAME AND WHETHER HIS FATHER WAS REALLY DEAD',
 'HE IS ALIVE SAID SHE BUT A LONG TIME AGO A RHINOCEROS CARRIED HIM OFF ON ITS HORN THEN THE BOY VOWED THAT HE WOULD GO IN SEARCH OF HIS FATHER AND MADE HIS MOTHER PUT HIM UP PROVISIONS FOR THE JOURNEY',
 'AND HE STARTED OFF TAKING WITH HIM AN IRON BOW AND A BIG BUNDLE OF ARROWS HE JOURNEYED ON ALL DAY AND AT NIGHTFALL HE CAME TO A VILLAGE THERE HE WENT UP TO THE HOUSE OF AN OLD WOMAN TO ASK FOR A BED',
 'GRANNIE GRANNIE OPEN THE DOOR I HAVE NO SON AND NO GRANDCHILDREN TO CALL ME GRANNIE GRUMBLED THE OLD WOMAN AND WENT TO OPEN THE DOOR TO SEE WHO WAS THERE AND WHEN SHE OPENED THE DOOR AND SAW HIM SHE SAID HO',
 'YOU ARE MY GRANDSON YES ANSWERED HE I AM YOUR GRANDCHILD SO SHE CALLED HIM INSIDE AND GAVE HIM A BED TO SLEEP ON THE OLD WOMAN WAS CALLED HUTIBUDI',
 'AND SHE 